In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr
import esmtools

In [2]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39669 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39669/status,
Dashboard: http://127.0.0.1:39669/status,Workers: 5
Total threads: 20,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39289,Workers: 5
Dashboard: http://127.0.0.1:39669/status,Total threads: 20
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:37991,Total threads: 4
Dashboard: http://127.0.0.1:33749/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:46863,


# load data

In [3]:
chunk_size = {"x":133,"y":320}

## Thermosteric

In [4]:
path = '/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/'

decomposition = 'thermosteric'

nexp = 'GAI'
sl_GAI = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_thermosteric_0_bottomprime.nc',chunks=chunk_size).sosteric
nexp = 'AI'
sl_AI = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_thermosteric_0_bottomprime.nc',chunks=chunk_size).sosteric
nexp = 'S'
sl_S = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_thermosteric_0_bottomprime.nc',chunks=chunk_size).sosteric

# GAI

In [5]:
sl_GAI_comp = (sl_GAI*1000).compute()


In [6]:
%%time
lin_GAI = esmtools.stats.linregress(sl_GAI_comp, dim="year")

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/esmtools/stats.py:394: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  results = xr.apply_ufunc(


CPU times: user 8min 54s, sys: 53.3 s, total: 9min 48s
Wall time: 8min 51s


In [7]:
diro = "/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/"

# convert into dataset and save : GAI -AI
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="thermosteric sea level - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 16 Feb 2024"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_thermosteric_0_bottom_trendGAI_linregress.nc")


# DIFF

In [5]:
%%time
sl_GAI_AI = ( (sl_GAI - sl_AI)*1000).compute()
sl_AI_S   = ( (sl_AI  - sl_S) *1000).compute()
sl_GAI_S  = ( (sl_GAI - sl_S) *1000).compute()

CPU times: user 836 ms, sys: 577 ms, total: 1.41 s
Wall time: 2.62 s


In [6]:
%%time
lin_GAI_AI = esmtools.stats.linregress(sl_GAI_AI, dim="year")
lin_AI_S   = esmtools.stats.linregress(sl_AI_S, dim="year")
lin_GAI_S  = esmtools.stats.linregress(sl_GAI_S, dim="year")

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/esmtools/stats.py:394: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  results = xr.apply_ufunc(


CPU times: user 32min 39s, sys: 2min 34s, total: 35min 13s
Wall time: 32min 38s


In [7]:
%%time

diro = "/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/"

# convert into dataset and save : GAI -AI
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI_AI.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_thermosteric_0_bottom_trendGAI_AI_linregress.nc")


# convert into dataset and save : AI -S
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_AI_S.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_thermosteric_0_bottom_trendAI_S_linregress.nc")


# convert into dataset and save : GAI -S
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI_S.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_thermosteric_0_bottom_trendGAI_S_linregress.nc")

CPU times: user 37.1 ms, sys: 60.4 ms, total: 97.5 ms
Wall time: 354 ms
